In [1]:
import torch
import autograd.numpy as np
from autograd import jacobian
from tensorflow.keras.layers import Softmax
from tensorflow.keras.config import set_floatx

In [2]:
set_floatx('float64')
torch.set_default_dtype(torch.float64)

# Errors 
Mean Percentage Error

In [3]:
def error(a_true: np.ndarray, a_pred: torch.Tensor) -> float:
    e = np.abs(a_true - a_pred.numpy()) / np.abs(a_true)
    return np.mean(e) * 100

# Softmax function

## one example/entry

In [4]:
Q = 5 # number of classes

Z = torch.randint(-20, 21, (1, Q)) / 2
Z

tensor([[ -8., -10.,  -3.,  10.,   4.]])

We will represent $\text{softmax}$ like $\sigma$.
$$
\sigma(\mathbf{z})_{j} = \frac{\exp (\mathbf{z})_{j}}{\sum_{k=1}^{Q} \exp (z_{k})}
$$
where $Q$ is the number of classes

In [5]:
SOFTMAX = Softmax()
tf_soft_1 = SOFTMAX(Z)
tf_soft_1.numpy()

array([[1.51922872e-08, 2.05605249e-09, 2.25473534e-06, 9.97525110e-01,
        2.47261754e-03]])

In [6]:
# our softmax funcion
def softmax_1(z: torch.Tensor) -> torch.Tensor:
    exp = torch.exp(z)
    return exp / exp.sum()

In [7]:
my_soft_1 = softmax_1(Z)
my_soft_1.numpy()

array([[1.51922872e-08, 2.05605249e-09, 2.25473534e-06, 9.97525110e-01,
        2.47261754e-03]])

In [8]:
# error
error(tf_soft_1.numpy(), my_soft_1)

1.7620633875561403e-14

## multiple examples/entrys

In [9]:
M = 100 # number of examples/entrys

Z = torch.randint(-20, 21, (M, Q)) / 2
Z.shape

torch.Size([100, 5])

$$
\mathbf{Z} = \begin{bmatrix}
    \mathbf{z}_{1,:} \\
    \mathbf{z}_{2,:} \\
    \vdots \\
    \mathbf{z}_{M,:}
\end{bmatrix}
$$
then its softmax over each row is like
$$
\sigma(\mathbf{Z}) = \begin{bmatrix}
    \sigma(\mathbf{z}_{1,:}) \\
    \sigma(\mathbf{z}_{2,:}) \\
    \vdots \\
    \sigma(\mathbf{z}_{M,:}) \\
\end{bmatrix}
$$

In [10]:
tf_soft_2 = SOFTMAX(Z)
tf_soft_2.shape

TensorShape([100, 5])

In [11]:
# our function
def softmax_2(z: torch.Tensor) -> torch.Tensor:
    exp = torch.exp(z)
    return exp / exp.sum(dim=-1, keepdims=True)

my_soft_2 = softmax_2(Z)
my_soft_2.shape

torch.Size([100, 5])

In [12]:
error(tf_soft_2.numpy(), my_soft_2)

1.1598623514224193e-14

# Gradient

## derivative one softmax respect to one example

In [13]:
N_FEATURE = 3 # select one feature to derivative

Z = torch.randint(-20, 21, (1, Q)) / 2

def der_soft_1(z):
    exp = np.exp(z)
    return exp[0,N_FEATURE] / np.sum(exp)

gradient = jacobian(der_soft_1)
grad = gradient(Z.numpy())
print(grad.shape)
print(grad)

(1, 5)
[[-6.77838593e-02 -3.07738245e-06 -1.39712947e-10  6.90284415e-02
  -1.24150469e-03]]


$$
\frac{\partial \sigma(\mathbf{z})_{j}}{\partial \mathbf{z}}
\in \mathbb{R} \times (1 \times Q) \Leftrightarrow 1 \times Q
$$
because $\mathbf{z} \in 1 \times Q$ and $\sigma(\mathbf{z})_{j} \in \mathbb{R}$. <br>
Then its jacobian in **Numerator layout** is:
$$
\frac{\partial \sigma(\mathbf{z})_{j}}{\partial \mathbf{z}} =
\begin{bmatrix}
    \frac{\partial \sigma(\mathbf{z})_{j}}{\partial z_{1}} &
    \frac{\partial \sigma(\mathbf{z})_{j}}{\partial z_{2}} &
    \cdots &
    \frac{\partial \sigma(\mathbf{z})_{j}}{\partial z_{Q}}
\end{bmatrix}
$$
there are two different types of the derivatives:
1. $\frac{\partial \sigma(\mathbf{z})_{j}}{\partial z_{i=j}}$

2. $\frac{\partial \sigma(\mathbf{z})_{j}}{\partial z_{i\neq j}}$

First case:
$$
\frac{\partial \sigma(\mathbf{z})_{j}}{\partial z_{i=j}} = 
\sigma(\mathbf{z})_{j} (1 - \sigma(\mathbf{z})_j)
$$

Second case:
$$
\frac{\partial \sigma(\mathbf{z})_{j}}{\partial z_{i\neq j}} =
-\sigma(\mathbf{z})_{j} \sigma(\mathbf{z})_{i}
$$

Therefore:
$$
\frac{\partial \sigma(\mathbf{z})_{j}}{\partial \mathbf{z}} =
\begin{bmatrix}
    -\sigma(\mathbf{z})_{j} \sigma(\mathbf{z})_{1} &
    \cdots &
    \sigma(\mathbf{z})_{j}(1 - \sigma(\mathbf{z})_j) &
    \cdots &
    -\sigma(\mathbf{z})_{j} \sigma(\mathbf{z})_{Q}
\end{bmatrix}
$$
or as vectorized form:
$$
\frac{\partial \sigma(\mathbf{z})_{j}}{\partial \mathbf{z}} =
\sigma(\mathbf{z})_j \odot
\begin{bmatrix}
    -\sigma(\mathbf{z})_{1} &
    \cdots &
    1 - \sigma(\mathbf{z})_{j} &
    \cdots &
    -\sigma(\mathbf{z})_{Q}
\end{bmatrix}
$$

In [14]:
def my_der_soft_1(z, j):
    soft = softmax_1(z)
    soft_j = soft[0,j].item()
    soft *= -1
    soft[0,j] += 1
    return soft_j * soft

my_grad = my_der_soft_1(Z, N_FEATURE)
my_grad.numpy()

array([[-6.77838593e-02, -3.07738245e-06, -1.39712947e-10,
         6.90284415e-02, -1.24150469e-03]])

In [15]:
error(grad, my_grad)

1.0266523470111423e-14

## derivative of multiple softmax respect to one example/entry

In [16]:
try:
    del gradient, grad
except: pass

In [17]:
Z = torch.randint(-20, 21, (1, Q)) / 2

def der_soft_2(z):
    exp = np.exp(z)
    return exp / np.sum(exp)

gradient = jacobian(der_soft_2)
grad = gradient(Z.numpy())
print(grad.shape)
grad

(1, 5, 1, 5)


array([[[[ 2.12703216e-01, -8.58746885e-05, -5.71188305e-02,
          -1.55265079e-01, -2.33431605e-04]],

        [[-8.58746885e-05,  2.79756608e-04, -5.20856315e-05,
          -1.41583426e-04, -2.12862071e-07]],

        [[-5.71188305e-02, -5.20856315e-05,  1.51485530e-01,
          -9.41730309e-02, -1.41583426e-04]],

        [[-1.55265079e-01, -1.41583426e-04, -9.41730309e-02,
           2.49964557e-01, -3.84863653e-04]],

        [[-2.33431605e-04, -2.12862071e-07, -1.41583426e-04,
          -3.84863653e-04,  7.60091546e-04]]]])

$$
\frac{\partial \sigma(\mathbf{z})}{\partial \mathbf{z}} \in
(1 \times Q) \times (1 \times Q)
$$
but to simplify we will use an easier notation:
$$
\Rightarrow
\frac{\partial \sigma(\mathbf{z})}{\partial \mathbf{z}} \in 
Q \times Q
$$
we will ignore the 1's axes for now. <br>
The derivative is like:
$$
\begin{align*}
\frac{\partial \sigma(\mathbf{z})}{\partial \mathbf{z}} &=
\begin{bmatrix}
    \frac{\partial \sigma(\mathbf{z})_{1}}{\partial \mathbf{z}} \\
    \frac{\partial \sigma(\mathbf{z})_{2}}{\partial \mathbf{z}} \\
    \vdots \\
    \frac{\partial \sigma(\mathbf{z})_{Q}}{\partial \mathbf{z}}
\end{bmatrix} \\
&= \begin{bmatrix}
    \frac{\partial \sigma(\mathbf{z})_{1}}{\partial z_{1}} &
    \frac{\partial \sigma(\mathbf{z})_{1}}{\partial z_{2}} &
    \cdots &
    \frac{\partial \sigma(\mathbf{z})_{1}}{\partial z_{Q}} \\
    \frac{\partial \sigma(\mathbf{z})_{2}}{\partial z_{1}} &
    \frac{\partial \sigma(\mathbf{z})_{2}}{\partial z_{2}} &
    \cdots &
    \frac{\partial \sigma(\mathbf{z})_{2}}{\partial z_{Q}} \\
    \vdots & \vdots & \ddots & \vdots \\
    \frac{\partial \sigma(\mathbf{z})_{Q}}{\partial z_{1}} &
    \frac{\partial \sigma(\mathbf{z})_{Q}}{\partial z_{2}} &
    \cdots &
    \frac{\partial \sigma(\mathbf{z})_{Q}}{\partial z_{Q}} \\
\end{bmatrix}
\end{align*}
$$
then using this derivatives:
$$
\begin{align*}
\frac{\partial \sigma(\mathbf{z})_{j}}{\partial z_{i=j}} &= 
\sigma(\mathbf{z})_{j} (1 - \sigma(\mathbf{z})_j) \\
\frac{\partial \sigma(\mathbf{z})_{j}}{\partial z_{i\neq j}} &=
-\sigma(\mathbf{z})_{j} \sigma(\mathbf{z})_{i}
\end{align*}
$$
therefore:
$$
\frac{\partial \sigma(\mathbf{z})}{\partial \mathbf{z}} =
\begin{bmatrix}
    \sigma(\mathbf{z})_{1} (1 - \sigma(\mathbf{z})_1) &
    -\sigma(\mathbf{z})_{1} \sigma(\mathbf{z})_{2} &
    \cdots &
    -\sigma(\mathbf{z})_{1} \sigma(\mathbf{z})_{Q} \\
    -\sigma(\mathbf{z})_{2} \sigma(\mathbf{z})_{1} &
    \sigma(\mathbf{z})_{2} (1 - \sigma(\mathbf{z})_2) &
    \cdots &
    -\sigma(\mathbf{z})_{2} \sigma(\mathbf{z})_{Q} \\
    \vdots & \vdots & \ddots & \vdots \\
    -\sigma(\mathbf{z})_{Q} \sigma(\mathbf{z})_{1} &
    -\sigma(\mathbf{z})_{Q} \sigma(\mathbf{z})_{2} &
    \cdots &
    \sigma(\mathbf{z})_{Q} (1 - \sigma(\mathbf{z})_Q)
\end{bmatrix}
$$
or as vectorized form:
$$
\frac{\partial \sigma(\mathbf{z})}{\partial \mathbf{z}} =
\text{diag}(\sigma(\mathbf{z})) - \sigma(\mathbf{z}) \sigma(\mathbf{z})^T
$$

In [18]:
def my_der_soft_2(z):
    soft = softmax_1(z)[0,:] # is necessary for Pytorch to work
    return torch.diag(soft) - torch.outer(soft, soft)

my_grad = my_der_soft_2(Z)
my_grad.numpy()

array([[ 2.12703216e-01, -8.58746885e-05, -5.71188305e-02,
        -1.55265079e-01, -2.33431605e-04],
       [-8.58746885e-05,  2.79756608e-04, -5.20856315e-05,
        -1.41583426e-04, -2.12862071e-07],
       [-5.71188305e-02, -5.20856315e-05,  1.51485530e-01,
        -9.41730309e-02, -1.41583426e-04],
       [-1.55265079e-01, -1.41583426e-04, -9.41730309e-02,
         2.49964557e-01, -3.84863653e-04],
       [-2.33431605e-04, -2.12862071e-07, -1.41583426e-04,
        -3.84863653e-04,  7.60091546e-04]])

In [19]:
error(grad[0,:,0,:], my_grad)

7.497936617049648e-15

## derivative of multiple softmax respect to multiple examples

### Problem statement

$$
\mathbf{Z} \in M \times Q
$$
where $M$ is the number of examples. Then softmax function is:
$$
\sigma(\mathbf{Z}) = \begin{bmatrix}
    \sigma(\mathbf{Z}_{1,:}) \\
    \sigma(\mathbf{Z}_{2,:}) \\
    \vdots \\
    \sigma(\mathbf{Z}_{M,:})
\end{bmatrix} \in M \times Q
$$
where
$$
\mathbf{Z}_{i,:} = \begin{bmatrix}
    Z_{i1} & Z_{i2} & \cdots & Z_{iQ}
\end{bmatrix} \in 1 \times Q
$$
for all $i = 1, ..., M$. Therefore
$$
\sigma(\mathbf{Z}_{i,:}) = \begin{bmatrix}
    \sigma(\mathbf{Z}_{i,:})_{1} & 
    \sigma(\mathbf{Z}_{i,:})_{2} & 
    \cdots & 
    \sigma(\mathbf{Z}_{i,:})_{Q}
\end{bmatrix} \in 1 \times Q
$$

### derivative

In [20]:
try:
    del gradient, grad
except: pass

In [21]:
Z = torch.randint(-20, 21, (M, Q)) / 2

def der_soft_3(z):
    exp = np.exp(z)
    return exp / np.sum(exp, axis=-1, keepdims=True)

gradient = jacobian(der_soft_3)
grad = gradient(Z.numpy())
print(grad.shape)

(100, 5, 100, 5)


$$
\frac{\mathrm{d} {\color{cyan}\sigma(\mathbf{Z})}}{\mathrm{d} {\color{Orange}\mathbf{Z}}} \in {\color{cyan}(M \times Q)} \times {\color{Orange}(M \times Q)}
$$
where
$$
\frac{\mathrm{d} {\color{cyan}\sigma(\mathbf{Z})_{pq}}}{\mathrm{d} {\color{Orange}\mathbf{Z}_{ij}}} \in {\color{cyan}(1 \times 1)} \times {\color{Orange}(1 \times 1)}
$$
therefore, the last derivative is:
$$
\frac{\partial \sigma(\mathbf{Z})_{pq}}{\partial \mathbf{Z}_{ij}} =
\begin{cases}
    0 & \text{ if } p\neq i \\ 
    \sigma(\mathbf{Z})_{pq}(1 - \sigma(\mathbf{Z})_{ij}) & \text{ if } p=i, q=j \\ 
    -\sigma(\mathbf{Z})_{pq} \sigma(\mathbf{Z})_{ij} & \text{ if } p=i, q\neq j 
\end{cases}
$$
for all $p,i = 1, ..., M$ and $q,j = 1, .., Q$. The last 2 cases looks so similar to past derivative $\frac{\partial \sigma(\mathbf{z})_{j}}{\partial \mathbf{z}}$

In [22]:
def my_der_soft_3(z):
    m, class_ = z.shape
    s = softmax_2(z)
    der = torch.zeros((m, class_, m, class_))
    for p in range(m):
        for q in range(class_):
            for i in range(m):
                for j in range(class_):
                    if p != i: continue
                    if q == j:
                        der[p,q,i,j] = s[p,q] * (1 - s[p,q])
                    else:
                        der[p,q,i,j] = -s[p,q] * s[i,j]
    return der

my_grad = my_der_soft_3(Z)
my_grad.shape

torch.Size([100, 5, 100, 5])

In [23]:
# + 1e-100 to avoid division by zero
error(grad + 1e-100, my_grad + 1e-100)

1.3353362552065126e-14

This solution is too slow, its complexity is $O(M^2Q^2) \approx O(n^4)$, but we can observe some similarities between this derivative and a past derivative, like:
$$
\frac{\partial \sigma(\mathbf{Z})_{p,:}}{\partial \mathbf{Z}_{i=p,:}}
\approx \frac{\partial \sigma(\mathbf{z})}{\partial \mathbf{z}}
$$
where
$$
\frac{\partial {\color{cyan}\sigma(\mathbf{Z})_{p,:}}}{\partial {\color{Orange}\mathbf{Z}_{i,:}}} \in {\color{cyan}(1 \times Q)} \times {\color{Orange}(1 \times Q)}
$$
**Remark**: yes we need 1's axes. <br>
Then we have 2 cases:
1. $\frac{\partial \sigma(\mathbf{Z})_{p,:}}{\partial \mathbf{Z}_{i=p,:}}$

2. $\frac{\partial \sigma(\mathbf{Z})_{p,:}}{\partial \mathbf{Z}_{i\neq p,:}}$

First case:
$$
\frac{\partial \sigma(\mathbf{Z})_{p,:}}{\partial \mathbf{Z}_{i=p,:}} = \text{diag}(\sigma(\mathbf{Z}_{p,:})) - \sigma(\mathbf{Z}_{p,:}) \sigma(\mathbf{Z}_{p,:})^T
$$

Second case:
$$
\frac{\partial \sigma(\mathbf{Z})_{p,:}}{\partial \mathbf{Z}_{i\neq p,:}} = \mathbf{0}
$$

In [24]:
def my_der_soft_4(z):
    m, class_ = z.shape
    der = torch.zeros((m,class_,m,class_))
    for i in range(m):
        der[i,:,i,:] = my_der_soft_2(z[None,i,:])
    return der

my_grad = my_der_soft_4(Z)
my_grad.shape

torch.Size([100, 5, 100, 5])

In [25]:
# + 1e-100 to avoid division by zero
error(grad + 1e-100, my_grad + 1e-100)

1.468651931025567e-14

In [26]:
# Vectorized form
def my_der_soft_5(z):
    m, n_classes = z.shape
    soft = softmax_2(z)

    diag_soft = torch.diag_embed(soft)
    outer_soft = torch.einsum('ij,ik->ijk', soft, soft)
    return (diag_soft - outer_soft).unsqueeze(2).expand(-1, -1, m, -1)

my_grad = my_der_soft_5(Z)
my_grad.shape

torch.Size([100, 5, 100, 5])

In [27]:
error(grad + 1e-100, my_grad + 1e-100)

1.562481426712466e+100

# Appendix A

$$
\begin{align*}
\sigma(\mathbf{z})_{j} &= \frac{\exp(\mathbf{z})_{j}}{\sum_{k=1}^{Q}\exp(z_k)} \\
&= \frac{\exp(\mathbf{z})_{j}}{\exp(z_1) + ... + \exp(z_j) + ... + \exp(z_Q)}
\end{align*}
$$
then, when $i = j$:
$$
\begin{align*}
\frac{\partial \sigma(\mathbf{z})_{j}}{\partial z_{i=j}} &=
\frac{\exp(\mathbf{z})_{j}(\sum_{k=1}^{Q}\exp(z_k)) - \exp(\mathbf{z})_{j}^{2}}
{(\sum_{k=1}^{Q}\exp(z_k))^{2}} \\
&= \frac{\exp(\mathbf{z})_{j}(\sum_{k=1}^{Q}\exp(z_k) - \exp(\mathbf{z})_{j})}
{(\sum_{k=1}^{Q}\exp(z_k))^{2}} \\
&= \frac{\exp(\mathbf{z})_{j}}{\sum_{k=1}^{Q}\exp(z_k)}
\left( 
    \frac{\sum_{k=1}^{Q}\exp(z_k) - \exp(\mathbf{z})_{j}}
    {\sum_{k=1}^{Q}\exp(z_k)}
\right) \\
&= \sigma(\mathbf{z})_j \left(
    \frac{\sum_{k=1}^{Q}\exp(z_k)}{\sum_{k=1}^{Q}\exp(z_k)} -
    \frac{\exp(\mathbf{z})_{j}}{\sum_{k=1}^{Q}\exp(z_k)}
\right) \\
&= \sigma(\mathbf{z})_j (1 - \sigma(\mathbf{z})_j)
\end{align*}
$$
when $i \neq j$:
$$
\begin{align*}
\frac{\partial \sigma(\mathbf{z})_{j}}{\partial z_{i\neq j}} &=
- \frac{\exp(\mathbf{z})_{j} \exp(\mathbf{z})_i}{(\sum_{k=1}^{Q}\exp(z_k))^{2}} \\
&= - \frac{\exp(\mathbf{z})_j}{\sum_{k=1}^{Q}\exp(z_k)} \left(
    \frac{\exp(\mathbf{z})_i}{\sum_{k=1}^{Q}\exp(z_k)}
\right) \\
&= - \sigma(\mathbf{z})_j \sigma(\mathbf{z})_i
\end{align*}
$$